## Create a system mixing SMIRNOFF and non-SMIRNOFF-formatted force fields

### Parametrize a molecule with the Sage force field

First, we parametrize the ligand (toluene) with the SMIRNOFF-format Sage force field through the usual route to create an OpenMM `System`.

In [1]:
from openff.toolkit import ForceField, Molecule, Topology
from openff.toolkit.utils import get_data_file_path

In [2]:
%env INTERCHANGE_EXPERIMENTAL=1

env: INTERCHANGE_EXPERIMENTAL=1


In [3]:
toluene_topology = Topology.from_pdb(
    get_data_file_path("molecules/toluene.pdb"),
    unique_molecules=[Molecule.from_smiles("Cc1ccccc1")],
)

sage = ForceField("openff-2.0.0.offxml")

toluene = sage.create_interchange(toluene_topology)

In [4]:
ff14sb = ForceField("ff14sb_off_impropers_0.0.3.offxml")

# The toolkit can load multi-component PDB files! See the docstring for some limitations:
# https://docs.openforcefield.org/projects/toolkit/en/latest/api/generated/openff.toolkit.topology.Topology.html#openff.toolkit.topology.Topology.from_pdb
t4_lysozyme_topology = Topology.from_pdb(
    get_data_file_path("systems/test_systems/T4_lysozyme_water_ions.pdb")
)
t4_lysozyme_topology.n_molecules

8885

In [5]:
# Interchange is still slow for some large systems, so for this example let's just slice out the protein
# (the 0th molecule in the topology) and create an Interchange for that. Optimizations are coming!
t4_lysozyme = ff14sb.create_interchange(t4_lysozyme_topology.molecule(0).to_topology())
t4_lysozyme

Interchange with 7 collections, non-periodic topology with 2634 atoms.

In [6]:
complex_system = t4_lysozyme + toluene

/Users/mattthompson/mambaforge/envs/openff-toolkit-test/lib/python3.10/site-packages/openff/interchange/components/interchange.py:630: UserWarning: Interchange object combination is experimental and likely to produce strange results. Any workflow using this method is not guaranteed to be suitable for production. Use with extreme caution and thoroughly validate results!
  warnings.warn(


In [7]:
complex_system.to_openmm(combine_nonbonded_forces=True)

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x110eab840> >